#**Purpose**

 The main purpose of this notebook is to do a short study of a use case in NLP.
 Data processing

*   Data processing (clean, tokenization ...)
*   Vectorization
*   Plot ome usefull information
*   Use of pipline
*   Machine learning algorithms
*   (...)







# **Library Installation**

This section is ised to install libraries

In [1]:
!pip install datasets 
!pip install spacy
!pip install folium==0.2.1
!pip install plotly
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Imports**
We strat by importing the libs that I'll use

In [2]:
import pandas as pd
import numpy as np
import datasets
import spacy

from spacy.lang.en import English
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn  import svm 

import string

from xgboost import XGBClassifier
import seaborn as sns

spacy.prefer_gpu()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


False

# **Load data**
In this section I load the data I want to use. In this case I use '**banking77**' datatset
Dataset composed of online banking queries annotated with their corresponding intents.

BANKING77 dataset provides a very fine-grained set of intents in a banking domain. It comprises 13,083 customer service queries labeled with 77 intents. It focuses on fine-grained single-domain intent detection.

For more information : https://huggingface.co/datasets/banking77


This datset is diveded to train and its, and test and its label.
I extract the train data and test data, then I put each one in a dataframe. Then I create third dataframe which conatin the all the data of banking 77.

In [3]:
dataset = datasets.load_dataset("banking77")

# Train data about 10000 text
data_train = pd.DataFrame()
data_train['text'] = dataset['train']['text']
#data_train['text'] = data_train['text'].apply(lambda x : x.lower())
#data_train['text'] = data_train['text'].astype(str)
data_train['label'] = dataset['train']['label']
data_train['data_type'] = 'train'

# Train data about 3000 text
data_test = pd.DataFrame()
data_test['text'] = dataset['test']['text']
#data_test['text'] = data_test['text'].apply(lambda x : x.lower())
#data_test['text'] = data_test['text'].astype(str)
data_test['label'] = dataset['test']['label']
data_test['data_type'] = 'test'

# All data about 13000 text
df = pd.DataFrame()
df = pd.concat([data_train, data_test], ignore_index=True)
df

Using custom data configuration default
Reusing dataset banking77 (/root/.cache/huggingface/datasets/banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)


  0%|          | 0/2 [00:00<?, ?it/s]

,text,label,data_type
0,I am still waiting on my card?,11,train
1,What can I do if my card still hasn't arrived ...,11,train
2,I have been waiting over a week. Is the card s...,11,train
3,Can I track my card while it is in the process...,11,train
4,"How do I know if I will get my card, or if it ...",11,train
...,...,...,...
13078,"If i'm not in the UK, can I still get a card?",24,test
13079,How many countries do you support?,24,test
13080,What countries do you do business in?,24,test
13081,What are the countries you operate in.,24,test


# **Data processing**

In this section I'll do the data processing.

## Some verification
I'll start by checkng if there is a NaN or double item in the dataset

In [4]:
# Check if dataset contain NaN Values
print(f"Is there NaN values in the dataset ? {df.isnull().values.any()}")

Is there NaN values in the dataset ? False


Great !! 
No NaN value in the data frame, this is a good news !

In [5]:
# Check if dataset contain the same text more than one occurence
print(f"Is there more than one occurence of text in the dataset ? {df['text'].duplicated().any()}")

Is there more than one occurence of text in the dataset ? False


Nice, all the texts in the dataset have one occurence, this is an other good news !

# Some information about the Dataset

In [6]:
# Count number of classes in te dataset
print(f"The dataset contain {df['label'].nunique()} classes")

The dataset contain 77 classes


In [7]:
# Calculate the length of each text in the dataset
df['text_character_nbr'] = df['text'].apply(lambda x : len(x)) 
df.head()

,text,label,data_type,text_character_nbr
0,I am still waiting on my card?,11,train,30
1,What can I do if my card still hasn't arrived ...,11,train,60
2,I have been waiting over a week. Is the card s...,11,train,58
3,Can I track my card while it is in the process...,11,train,59
4,"How do I know if I will get my card, or if it ...",11,train,54


In [8]:
# Calculate mean length of each text in the dataset
print(f"The mean length of each text in the dataset : {df['text_character_nbr'].mean()}")

The mean length of each text in the dataset : 58.23985324466865


In [9]:
print(f"The shorter texts in the dataset {df['text_character_nbr'].min()}")
df.loc[df['text'].str.len() == df['text_character_nbr'].min()]

The shorter texts in the dataset 13


,text,label,data_type,text_character_nbr
2294,Can I top up?,60,train,13
4003,The card PIN?,38,train,13
4046,wheres my pin,38,train,13
4889,Lost password,44,train,13
5313,phone is gone,42,train,13
8547,Fix my top up,59,train,13
11272,I need my PIN,38,test,13


In [10]:
print(f"The longer texts in the dataset {df['text_character_nbr'].max()}")
df.loc[df['text'].str.len() == df['text_character_nbr'].max()]

The longer texts in the dataset 433


,text,label,data_type,text_character_nbr
3917,Hearing back from us regarding your important ...,68,train,433


# Data processing

Here I'll ro the processing of the data (tokenization, remove stopwords ...
In this section I decided to use the scapy libary.

SpaCy is a Python language processing library developed by Matt Honnibal and Ines Montani.
This library allows to perform tokenization analysis and named entity recognition operations on texts in more than 50 languages.

For more information : https://spacy.io/

In [11]:
# Load small english model: https://spacy.io/models
nlp = spacy.load("en_core_web_sm")

def spacy_tokenizer(sentence):   

  _sentence = nlp(sentence)

  # Removing stop words
  mytokens = [str(word.text).lower() for word in _sentence if word.is_stop != True and word.is_punct != True and word.is_space != True]

  # return preprocessed list of tokens
  return mytokens


The variable '**doc**' is a sequence of tokens that contains not just the original text but all the results produced by the spaCy model after processing the text. Useful information such as the lemma of the text, whether it is a stop word or not, named entities, the word vector of the text and so on are pre-computed and readily stored in the Doc object.

Now I'll add the raw text tokenized to the data frame, then I'll do somme treatment like removing stopwords, punctuation ...

In [12]:
df['text_tokenized'] = df['text'].apply(spacy_tokenizer)
df['word_nbr'] = df['text_tokenized'].apply(lambda x: len(x))
df

,text,label,data_type,text_character_nbr,text_tokenized,word_nbr
0,I am still waiting on my card?,11,train,30,"[waiting, card]",2
1,What can I do if my card still hasn't arrived ...,11,train,60,"[card, arrived, 2, weeks]",4
2,I have been waiting over a week. Is the card s...,11,train,58,"[waiting, week, card, coming]",4
3,Can I track my card while it is in the process...,11,train,59,"[track, card, process, delivery]",4
4,"How do I know if I will get my card, or if it ...",11,train,54,"[know, card, lost]",3
...,...,...,...,...,...,...
13078,"If i'm not in the UK, can I still get a card?",24,test,45,"[uk, card]",2
13079,How many countries do you support?,24,test,34,"[countries, support]",2
13080,What countries do you do business in?,24,test,37,"[countries, business]",2
13081,What are the countries you operate in.,24,test,38,"[countries, operate]",2


# The 100 most frequent words

## The first case without any treatment

In [13]:
word_freq = pd.Series(np.concatenate([x for x in df.text_tokenized])).value_counts()
df_word_freq = pd.DataFrame(word_freq).reset_index().rename(columns={'index': 'Word', 0:'Frequency'})
fig = px.scatter(df_word_freq[:100], x='Word', y='Frequency', template='plotly_white',
                 title='Frequency of word in the dataset', color='Frequency', 
                 hover_name='Word')
fig.update_traces(marker=dict(line=dict(width=1, color='Gray')))
fig.show()

# The 100 most proper noun

In [14]:
df['proper_noun'] = df['text'].apply(lambda x: [token.text for token in nlp(x) if token.pos_ == 'PROPN'])
propn_freq = pd.Series(np.concatenate([x for x in df.proper_noun])).value_counts()
df_propn_freq = pd.DataFrame(propn_freq).reset_index().rename(columns={'index': 'Proper noun', 0:'Frequency'})

fig = px.scatter(df_propn_freq[:100], x='Proper noun', y='Frequency', template='plotly_white',
                 title='Frequency of proper noun in the dataset', color='Frequency', 
                 hover_name='Proper noun')
fig.update_traces(marker=dict(line=dict(width=1, color='Gray')))
fig.show()

In [15]:
df

,text,label,data_type,text_character_nbr,text_tokenized,word_nbr,proper_noun
0,I am still waiting on my card?,11,train,30,"[waiting, card]",2,[]
1,What can I do if my card still hasn't arrived ...,11,train,60,"[card, arrived, 2, weeks]",4,[]
2,I have been waiting over a week. Is the card s...,11,train,58,"[waiting, week, card, coming]",4,[]
3,Can I track my card while it is in the process...,11,train,59,"[track, card, process, delivery]",4,[]
4,"How do I know if I will get my card, or if it ...",11,train,54,"[know, card, lost]",3,[]
...,...,...,...,...,...,...,...
13078,"If i'm not in the UK, can I still get a card?",24,test,45,"[uk, card]",2,[UK]
13079,How many countries do you support?,24,test,34,"[countries, support]",2,[]
13080,What countries do you do business in?,24,test,37,"[countries, business]",2,[]
13081,What are the countries you operate in.,24,test,38,"[countries, operate]",2,[]


# **Machine Learning part**

For this part, I'll start by vectorizing the data, then I'll use SGDClassifier to predict the class for the data_test.
I'll also show, some metrics to mesure the performance of the SGDClassifeir

In [16]:
# TF-IDF Vectorization
vect_tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer)

# Vectorization of the data train
X_train_transformed = vect_tfidf.fit_transform(df['text'].loc[df['data_type'] == 'train'])
y_train = df['label'].loc[df['data_type'] == 'train']

# Vectorization of the data test
X_test_transformed = vect_tfidf.transform(df['text'].loc[df['data_type'] == 'test'])
y_test = df['label'].loc[df['data_type'] == 'test']

In [17]:
# Use SGDClassifier
clf= SGDClassifier()
clf.fit(X_train_transformed, y_train)
predict = clf.predict(X_test_transformed)
accuracy_score(y_test, predict)

0.827922077922078

In [18]:
clf_report = classification_report(y_test, predict)
print(f'The calssification \n{clf_report}')

The calssification 
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        40
           1       0.83      0.97      0.90        40
           2       0.97      0.97      0.97        40
           3       0.92      0.88      0.90        40
           4       0.93      0.95      0.94        40
           5       0.71      0.72      0.72        40
           6       0.86      0.90      0.88        40
           7       0.94      0.85      0.89        40
           8       0.85      0.97      0.91        40
           9       0.98      1.00      0.99        40
          10       0.85      0.42      0.57        40
          11       0.87      0.82      0.85        40
          12       0.71      0.85      0.77        40
          13       0.87      1.00      0.93        40
          14       0.53      0.82      0.65        40
          15       0.85      0.88      0.86        40
          16       0.69      0.72      0.71        40
       

**For this section I'll use a pipline**

A machine learning pipeline is a way to codify and automate the workflow it takes to produce a machine learning model. Machine learning pipelines consist of multiple sequential steps that do everything from data extraction and preprocessing to model training and deployment.

For this first pipline I'll use as Stochastic Gradient Descent Clasiifier (SGDClassifier) from Scikit Learn library.
For the vectorization I'll use TFIDF (term frequency-inverse document frequency)

In [19]:
# Definition of pipline
class DataProcessing(TransformerMixin):
  def transform(self, X, **transform_params):
      # Cleaning Text
      print('done')
      return [spacy_tokenizer(text) for text in X]

  def fit(self, X, y=None, **fit_params):
      return self

  def get_params(self, deep=True):
      return {}

In [20]:
sgd_clf_pipe = Pipeline([("cleaner", DataProcessing),
                         ('tfidf_vect', TfidfVectorizer()),
                         ('clf', SGDClassifier)])